# 0. Install Dependencies

In [19]:
""" !pip install tensorflow
!pip install gym
!pip install keras
!pip install keras-rl2 """

' !pip install tensorflow\n!pip install gym\n!pip install keras\n!pip install keras-rl2 '

source: https://github.com/nicknochnack/OpenAI-Reinforcement-Learning-with-Custom-Environment/blob/main/OpenAI%20Custom%20Environment%20Reinforcement%20Learning.ipynb

# 1. Test Random Environment with OpenAI Gym

In [20]:
from gym import Env
from gym.spaces import Discrete, Box
import numpy as np
import random

In [21]:
class ShowerEnv(Env):
    def __init__(self):
        # Actions we can take, down, stay, up
        self.action_space = Discrete(3)
        # Temperature array
        self.observation_space = Box(low=np.array([0]), high=np.array([100]))
        # Set start temp
        self.state = 38 + random.randint(-3,3)
        # Set shower length
        self.shower_length = 60
        
    def step(self, action):
        # Apply action
        # 0 -1 = -1 temperature
        # 1 -1 = 0 
        # 2 -1 = 1 temperature 
        self.state += action -1 
        # Reduce shower length by 1 second
        self.shower_length -= 1 
        
        # Calculate reward
        if self.state >=37 and self.state <=39: 
            reward =1 
        else: 
            reward = -1 
        
        # Check if shower is done
        if self.shower_length <= 0: 
            done = True
        else:
            done = False
        
        # Apply temperature noise
        self.state += random.randint(-1,1)
        # Set placeholder for info
        info = {}
        
        # Return step information
        return self.state, reward, done, info

    def render(self):
        # Implement viz
        pass
    
    def reset(self):
        # Reset shower temperature
        self.state = 38 + random.randint(-3,3)
        # Reset shower time
        self.shower_length = 60 
        return self.state
    

In [22]:
env = ShowerEnv()

In [23]:
env.observation_space.sample()

array([42.352146], dtype=float32)

In [24]:
episodes = 10
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        #env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))

Episode:1 Score:-8
Episode:2 Score:6
Episode:3 Score:-38
Episode:4 Score:-40
Episode:5 Score:-14
Episode:6 Score:-48
Episode:7 Score:-60
Episode:8 Score:-60
Episode:9 Score:-36
Episode:10 Score:-14


# 2. Create a Deep Learning Model with Keras

In [25]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers.legacy import Adam

In [26]:
states = env.observation_space.shape
actions = env.action_space.n

In [27]:
def build_model(states, actions):
    model = Sequential()    
    model.add(Dense(24, activation='relu', input_shape=states))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [29]:
import tensorflow as tf

In [30]:

from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory
from tensorflow.keras.optimizers.legacy import Adam

In [31]:
model = build_model(states, actions)

In [32]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 24)                48        
                                                                 
 dense_1 (Dense)             (None, 24)                600       
                                                                 
 dense_2 (Dense)             (None, 3)                 75        
                                                                 
Total params: 723
Trainable params: 723
Non-trainable params: 0
_________________________________________________________________


# 3. Build Agent with Keras-RL

In [33]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, 
                  nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn

In [34]:
dqn = build_agent(model, actions)
#dqn.compile(optimizer=Adam(learning_rate=1e-3), metrics=['mae'])
dqn.compile(Adam(lr=1e-3), metrics=['mae'])
dqn.fit(env, nb_steps=50000, visualize=False, verbose=1)

C:\Users\darry\AppData\Roaming\Python\Python311\site-packages\keras\optimizers\legacy\adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Training for 50000 steps ...
Interval 1 (0 steps performed)


C:\Users\darry\AppData\Roaming\Python\Python311\site-packages\keras\engine\training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


   10/10000 [..............................] - ETA: 1:02 - reward: -1.0000   

c:\Users\darry\AppData\Local\Programs\Python\Python311\Lib\site-packages\rl\memory.py:37: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')


10000/10000 [==============================] - 198s 20ms/step - reward: -0.6290
166 episodes - episode_reward: -37.771 [-60.000, 22.000] - loss: 1.155 - mae: 6.719 - mean_q: -9.086

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 229s 23ms/step - reward: -0.6406
167 episodes - episode_reward: -38.383 [-60.000, 20.000] - loss: 2.317 - mae: 11.437 - mean_q: -16.385

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 242s 24ms/step - reward: -0.6436
167 episodes - episode_reward: -38.635 [-60.000, 20.000] - loss: 2.979 - mae: 13.017 - mean_q: -18.757

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 210s 21ms/step - reward: -0.5920
166 episodes - episode_reward: -35.506 [-60.000, 10.000] - loss: 3.069 - mae: 13.170 - mean_q: -18.993

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 223s 22ms/step - reward: -0.5920
done, took 1101.958 seconds


In [35]:
scores = dqn.test(env, nb_episodes=100, visualize=False)
print(np.mean(scores.history['episode_reward']))

Testing for 100 episodes ...
Episode 1: reward: -54.000, steps: 60
Episode 2: reward: -52.000, steps: 60
Episode 3: reward: -54.000, steps: 60
Episode 4: reward: -56.000, steps: 60
Episode 5: reward: -60.000, steps: 60
Episode 6: reward: -60.000, steps: 60
Episode 7: reward: -60.000, steps: 60
Episode 8: reward: -60.000, steps: 60
Episode 9: reward: -54.000, steps: 60
Episode 10: reward: -60.000, steps: 60
Episode 11: reward: -56.000, steps: 60
Episode 12: reward: -48.000, steps: 60
Episode 13: reward: -54.000, steps: 60
Episode 14: reward: -60.000, steps: 60
Episode 15: reward: -58.000, steps: 60
Episode 16: reward: -60.000, steps: 60
Episode 17: reward: -52.000, steps: 60
Episode 18: reward: -54.000, steps: 60
Episode 19: reward: -58.000, steps: 60
Episode 20: reward: -60.000, steps: 60
Episode 21: reward: -58.000, steps: 60
Episode 22: reward: -56.000, steps: 60
Episode 23: reward: -58.000, steps: 60
Episode 24: reward: -54.000, steps: 60
Episode 25: reward: -56.000, steps: 60
Episo

In [36]:
_ = dqn.test(env, nb_episodes=15, visualize=True)

Testing for 15 episodes ...


TypeError: ShowerEnv.render() got an unexpected keyword argument 'mode'

# 4. Reloading Agent from Memory

In [ ]:
dqn.save_weights('dqn_weights.h5f', overwrite=True)

In [ ]:
del model
del dqn
del env

In [ ]:
env = gym.make('CartPole-v0')
actions = env.action_space.n
states = env.observation_space.shape[0]
model = build_model(states, actions)
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])

In [ ]:
dqn.load_weights('dqn_weights.h5f')

In [ ]:
_ = dqn.test(env, nb_episodes=5, visualize=True)